In [140]:
import folium
import pandas as pd
import plotly
import matplotlib.pyplot as plt
import requests
import urllib.parse
from folium.plugins import FastMarkerCluster, MarkerCluster

def get_coord(address):
    '''Esta funcion busca las coordenadas de una dirección en openstreetmap.
    Los casos especiales se modificaron para que encontrara la direccion correcta
    '''
    if address=="Universidad Católica del Norte": address+=" Antofagasta"
    if address=="Universidad San Sebastián": address+=" Santiago"
    if address=="Universidad Nacional Andrés Bello": address="Universidad Andrés Bello"
    
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

    response = requests.get(url).json()
    return [response[0]["lat"],response[0]["lon"]]

In [132]:
data = pd.read_csv("Raw_data.csv", sep = ";", header=0)

#Filtrado por pago
univ = data.loc[(data["Pagado"]!="No") & (data["Pagado"]!="No Inscrito")].value_counts("Universidad")

#Alumnos Por Universidad
univ = univ.to_frame(name = "Pop")
univ.reset_index(level=0, inplace=True)

#Buscar coordenadas de cada Universidad
univ["Loc"] = univ["Universidad"].apply(get_coord)

In [154]:

# Crear mapa en folium
m = folium.Map(
    location = [0, -30],
    zoom_start=2.5,
    tiles='OpenStreetMap'
)

# Añadir marcadores agrupados
marker_cluster = folium.plugins.MarkerCluster().add_to(m)

# Añadir marcador e informacion de universidad y alumnos
for index,row in univ.iterrows():
    folium.Marker(location=row["Loc"],popup=f"{row['Universidad']}\n Estudiantes: {row['Pop'] }").add_to(marker_cluster)
m